In [1]:
import os

if "README.md" not in os.listdir():
    os.chdir("..")

print(os.listdir())

['sim_search.py', 'mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024_5k_valid_sdf_filter_smiles_simplified.csv', 'path_logP_QED.py', '.git', '__pycache__', 'GA_rediscover.py', 'mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024_5k_valid_sdf_filter.csv', 'stoned_selfies_tut.ipynb', 'generalized_path.py', 'notebooks', 'mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024_5k_valid_sdf_filter_smiles_simplified', 'realize_path.py', 'environment.yml', 'readme_docs', 'data', 'README.md']


---

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from selfies import encoder, decoder
from rdkit import Chem
from tqdm import tqdm

from typing import List, Dict

In [3]:
import sim_search

In [4]:
diffdock_results: pd.DataFrame = (
    pd.read_csv(Path("data") / "diffdock-results.csv")
    .drop(columns=["Unnamed: 0"])
    .rename(columns={"complex_name": "molecule_id", "ligand_description": "smiles"})
)
diffdock_results_ge0 = diffdock_results.loc[
    (diffdock_results["confidence"] > 0.0).values
]

In [5]:
diffdock_results_ge0

,molecule_id,confidence,smiles
0,CHEMBL486390,0.50,Oc1cc2c3c(c1)-c1cc4c(cc1CN3CC2)OCO4
1,CHEMBL3984788,0.40,O=C1C=C(N2CCC[C@H]2C(=O)O)C(=O)c2ccccc21
2,CHEMBL4087163,0.33,O=C1C[C@@H](c2cc(O)cc(O)c2)Oc2cc(O)ccc21
3,CHEMBL5194971,0.30,CCCCc1nc(C(=O)O)cc2c1[nH]c1ccccc12
4,CHEMBL35859,0.30,Oc1ccc([C@@H]2CN3CCC[C@@H]3c3ccccc32)cc1O
...,...,...,...
78,CHEMBL1365118,0.01,COC1=C(Nc2ccc(O)cc2)C(=O)c2ccccc2C1=O
79,CHEMBL3609294,0.01,COc1ccc2c(=O)c(O)c(-c3ccc(O)cc3)oc2c1
80,CHEMBL3604191,0.01,O=C1C=C(Oc2ccccc2)C(=O)c2c(O)cccc21
81,CHEMBL4289859,0.01,COc1cc2c3c(c1OC)-c1ccccc1C[C@@H]3N(C)CC2


In [6]:
len(set(diffdock_results_ge0["molecule_id"].values.tolist()))

83

---

In [7]:
# def get_mutated_df(
#     mol_df: pd.DataFrame, mutation_steps: int, num_mutations_per_step: int
# ) -> pd.DataFrame:
#     assert "smiles" in mol_df.columns
#     assert "molecule_id" in mol_df.columns

#     mol_df = mol_df.copy()
#     mol_df["canonical_smiles"] = (
#         mol_df["smiles"].apply(lambda smi: sim_search.sanitize_smiles(smi)[1]).values
#     )
#     mol_df["selfie"] = mol_df["canonical_smiles"].apply(encoder)

#     mutated_mol_df: pd.DataFrame = pd.DataFrame(
#         data={
#             "base_molecule_id": mol_df["molecule_id"].values,
#             "base_selfie": mol_df["selfie"].values,
#             "base_canonical_smiles": mol_df["canonical_smiles"].values,
#             "mutation_stage": [0] * len(mol_df),
#             "mutation_id": [0] * len(mol_df),
#             "parent_mutation_id": [None] * len(mol_df),
#             "selfie": mol_df["selfie"].values,
#             "canonical_smiles": mol_df["canonical_smiles"].values,
#         }
#     )

#     prev_mutations_df: pd.DataFrame = mutated_mol_df.copy(deep=True)

#     for i in tqdm(range(1, mutation_steps + 1)):
#         new_mutations_df: pd.DataFrame = pd.DataFrame(columns=prev_mutations_df.columns)
#         for j in range(num_mutations_per_step):
#             selfies_: List[str] = sim_search.get_mutated_SELFIES(
#                 selfies_ls=prev_mutations_df["selfie"].values, num_mutations=1
#             )
#             new_mutations_df = pd.concat(
#                 [
#                     new_mutations_df,
#                     pd.DataFrame(
#                         data={
#                             "base_molecule_id": prev_mutations_df[
#                                 "base_molecule_id"
#                             ].values,
#                             "base_selfie": prev_mutations_df["base_selfie"].values,
#                             "base_canonical_smiles": prev_mutations_df[
#                                 "base_canonical_smiles"
#                             ].values,
#                             "mutation_stage": [i] * len(prev_mutations_df),
#                             "mutation_id": [j] * len(prev_mutations_df),
#                             "parent_mutation_id": prev_mutations_df[
#                                 "mutation_id"
#                             ].values,
#                             "selfie": selfies_,
#                             "canonical_smiles": [
#                                 sim_search.sanitize_smiles(decoder(slf))[1]
#                                 for slf in selfies_
#                             ],
#                         }
#                     ),
#                 ],
#                 ignore_index=True,
#             )
#         mutated_mol_df = pd.concat(
#             [mutated_mol_df, new_mutations_df],
#             ignore_index=True,
#         ).copy(deep=True)
#         print(
#             len(
#                 mutated_mol_df.loc[~mutated_mol_df["parent_mutation_id"].isna().values]
#             ),
#             len(
#                 mutated_mol_df.loc[~mutated_mol_df["parent_mutation_id"].isna().values]
#                 .apply(
#                     lambda x: x["base_molecule_id"]
#                     + "-"
#                     + str(x["mutation_stage"])
#                     + "-"
#                     + str(x["mutation_id"])
#                     + "-"
#                     + str(int(x["parent_mutation_id"])),
#                     axis=1,
#                 )
#                 .unique()
#             ),
#         )
#         prev_mutations_df = new_mutations_df.copy(deep=True)

#     return mutated_mol_df

In [8]:
import pandas as pd
import numpy as np
from typing import List, Optional


def get_mutated_df(
    mol_df: pd.DataFrame,
    mutation_steps: int,
    num_mutations_per_step: int,
    encoder,  # presumably your SELFIE encoder
    decoder,  # presumably your SELFIE decoder
    sim_search,  # presumably your sim_search object
) -> pd.DataFrame:
    """
    Mutate molecules in multiple stages, ensuring each row has a unique integer row_id.
    No complicated lineage column needed: we simply store the parent's row_id.
    """

    # Basic validations
    assert "smiles" in mol_df.columns, "mol_df must have 'smiles' column."
    assert "molecule_id" in mol_df.columns, "mol_df must have 'molecule_id' column."

    # 1) Preprocess the base DataFrame
    mol_df = mol_df.copy()
    mol_df["canonical_smiles"] = mol_df["smiles"].apply(
        lambda s: sim_search.sanitize_smiles(s)[1]
    )
    mol_df["selfie"] = mol_df["canonical_smiles"].apply(encoder)

    # 2) Create the initial mutated_mol_df (stage=0)
    mutated_mol_df = pd.DataFrame(
        {
            "row_id": np.arange(len(mol_df)),  # unique integer row IDs
            "parent_row_id": [None] * len(mol_df),  # base molecules have no parent
            "base_molecule_id": mol_df["molecule_id"].values,
            "base_selfie": mol_df["selfie"].values,
            "base_canonical_smiles": mol_df["canonical_smiles"].values,
            "mutation_stage": [0] * len(mol_df),
            "mutation_id": [0] * len(mol_df),  # e.g. 0 for all base
            "selfie": mol_df["selfie"].values,
            "canonical_smiles": mol_df["canonical_smiles"].values,
        }
    )

    # We'll keep track of the next integer ID to assign
    next_row_id = len(mutated_mol_df)

    # We'll mutate from the stage-0 set
    prev_mutations_df = mutated_mol_df.copy(deep=True)

    # 3) Loop over stages
    for i in tqdm(range(1, mutation_steps + 1)):
        new_mutations_df = pd.DataFrame(columns=mutated_mol_df.columns)

        # For each sub-mutation "j" from 0..(num_mutations_per_step-1):
        for j in range(num_mutations_per_step):
            # Generate mutated selfies from the parent's SELFIE
            new_selfies: List[str] = sim_search.get_mutated_SELFIES(
                selfies_ls=prev_mutations_df["selfie"].values, num_mutations=1
            )

            # Build the child rows
            n_parents = len(prev_mutations_df)  # how many parents in the last stage
            child_df = pd.DataFrame(
                {
                    "row_id": np.arange(next_row_id, next_row_id + n_parents),
                    "parent_row_id": prev_mutations_df["row_id"].values,
                    "base_molecule_id": prev_mutations_df["base_molecule_id"].values,
                    "base_selfie": prev_mutations_df["base_selfie"].values,
                    "base_canonical_smiles": prev_mutations_df[
                        "base_canonical_smiles"
                    ].values,
                    "mutation_stage": [i] * n_parents,
                    "mutation_id": [j] * n_parents,
                    "selfie": new_selfies,
                    "canonical_smiles": [
                        sim_search.sanitize_smiles(decoder(slf))[1]
                        for slf in new_selfies
                    ],
                }
            )

            next_row_id += n_parents  # increment the global row counter
            new_mutations_df = pd.concat(
                [new_mutations_df, child_df], ignore_index=True
            )

        # Add all these new child rows to the big DataFrame
        mutated_mol_df = pd.concat(
            [mutated_mol_df, new_mutations_df], ignore_index=True
        )

        # The new children become the parents for the next stage
        prev_mutations_df = new_mutations_df.copy(deep=True)

    return mutated_mol_df

In [9]:
mutated_mol_df = get_mutated_df(
    mol_df=diffdock_results_ge0,
    mutation_steps=4,
    num_mutations_per_step=8,
    encoder=encoder,
    decoder=decoder,
    sim_search=sim_search,
)

100%|██████████| 4/4 [08:10<00:00, 122.62s/it]


In [10]:
mutated_mol_df.shape

(388523, 9)

In [10]:
# mutated_mol_df
mutated_mol_df = mutated_mol_df[~mutated_mol_df["parent_row_id"].isna().values]
mutated_mol_df["molecule_id"] = mutated_mol_df.apply(
    lambda x: x["base_molecule_id"]
    + "-"
    + str(x["mutation_stage"])
    + "-"
    + str(x["row_id"])
    + "-"
    + str(int(x["parent_row_id"])),
    axis=1,
)

In [11]:
mutated_mol_df["molecule_id"].value_counts()

molecule_id
CHEMBL1668346-4-388522-48554    1
CHEMBL486390-1-83-0             1
CHEMBL3984788-1-84-1            1
CHEMBL4087163-1-85-2            1
CHEMBL5194971-1-86-3            1
                               ..
CHEMBL254825-1-95-12            1
CHEMBL3422848-1-94-11           1
CHEMBL2104782-1-93-10           1
CHEMBL12117-1-92-9              1
CHEMBL338521-1-91-8             1
Name: count, Length: 388440, dtype: int64

In [12]:
mutated_mol_df_deduplicated = mutated_mol_df.drop_duplicates(subset="canonical_smiles")

In [13]:
from syba.syba import SybaClassifier

syba: SybaClassifier = SybaClassifier()
syba.fitDefaultScore()


def calculate_syba_score(mol_df: pd.DataFrame) -> pd.DataFrame:
    assert "canonical_smiles" in tqdm(mol_df.columns)
    mol_df["syba_score"] = [
        syba.predict(smi=x)
        for x in tqdm(mol_df["canonical_smiles"], desc="Calculating SYBA scores")
    ]
    return mol_df

The syba is calculated here as synthetitazability migh change after mutation.

In [14]:
mutated_mol_df_deduplicated_with_syba = calculate_syba_score(
    mutated_mol_df_deduplicated
)

Calculating SYBA scores:   0%|          | 112/281634 [00:00<09:43, 482.68it/s]

Calculating SYBA scores: 100%|██████████| 281634/281634 [01:38<00:00, 2848.78it/s]
/tmp/ipykernel_376284/3370714782.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mol_df["syba_score"] = [


In [15]:
def calculate_fp_similarities(
    mol_df: pd.DataFrame, fp_types: List[str]
) -> pd.DataFrame:
    mol_df = mol_df.copy()
    mol_df[fp_types] = np.nan

    unique_base_canonical_smiles = mol_df["base_canonical_smiles"].unique()

    for fp_type in fp_types:
        for base_canonical_smiles in tqdm(
            unique_base_canonical_smiles, desc=f"Calculating {fp_type}"
        ):
            mol_df.loc[
                (mol_df["base_canonical_smiles"] == base_canonical_smiles).values,
                fp_type,
            ] = sim_search.get_fp_scores(
                smiles_back=mol_df.loc[
                    (mol_df["base_canonical_smiles"] == base_canonical_smiles).values,
                    "canonical_smiles",
                ].values,
                target_smi=base_canonical_smiles,
                fp_type=fp_type,
            )

    return mol_df

In [16]:
fp_types: List[str] = [
    "AP",
    "PHCO",
    "BPF",
    "BTF",
    "PATH",
    "ECFP4",
    "ECFP6",
    "FCFP4",
    "FCFP6",
]


mutated_mol_df_deduplicated_with_syba_fp_sim = calculate_fp_similarities(
    mol_df=mutated_mol_df_deduplicated_with_syba, fp_types=fp_types
)

Calculating FCFP6: 100%|██████████| 83/83 [01:31<00:00,  1.10s/it]


In [17]:
mutated_mol_df_deduplicated_with_syba_fp_sim

,row_id,parent_row_id,base_molecule_id,base_selfie,base_canonical_smiles,mutation_stage,mutation_id,selfie,canonical_smiles,molecule_id,syba_score,AP,PHCO,BPF,BTF,PATH,ECFP4,ECFP6,FCFP4,FCFP6
83,83,0,CHEMBL486390,[O][C][=C][C][=C][C][=Branch1][Ring2][=C][Ring...,Oc1cc2c3c(c1)-c1cc4c(cc1CN3CC2)OCO4,1,0,[O][C][=C][C][=C][C][=Branch1][Ring2][=C][Ring...,OCOC1=C=Cc2cc3c4c(O)c2C(=C1)CN3CC4,CHEMBL486390-1-83-0,-90.927474,0.529661,0.121739,0.671296,0.625000,0.470328,0.239130,0.173228,0.280899,0.201613
84,84,1,CHEMBL3984788,[O][=C][C][=C][Branch1][=N][N][C][C][C][C][Rin...,O=C1C=C(N2CCCC2C(=O)O)C(=O)c2ccccc21,1,0,[O][=C][C][=C][Branch1][=N][C][C][C][C][Ring1]...,COC(=O)CCCCC(C=O)=CC=O,CHEMBL3984788-1-84-1,9.971354,0.185654,0.017778,0.313084,0.180000,0.089192,0.056180,0.045045,0.205128,0.160000
85,85,2,CHEMBL4087163,[O][=C][C][C][Branch1][#C][C][=C][C][Branch1][...,O=C1CC(c2cc(O)cc(O)c2)Oc2cc(O)ccc21,1,0,[#B-1][O][=C][C][C][Branch1][#C][C][=C][C][Bra...,[B-]OC1CC(c2cc(O)cc(O)c2)Oc2cc(O)ccc21,CHEMBL4087163-1-85-2,-8.366746,0.632653,0.639498,0.818182,0.680000,0.677309,0.619718,0.510417,0.796875,0.705882
86,86,3,CHEMBL5194971,[C][C][C][C][C][=N][C][Branch1][=Branch1][C][=...,CCCCc1nc(C(=O)O)cc2c1[nH]c1ccccc12,1,0,[C][C][C][C][=N][C][Branch1][=Branch1][C][=Bra...,CCCc1nc(C(=O)O)cc2c1[nH]c1ccccc12,CHEMBL5194971-1-86-3,46.492873,0.823232,0.687831,0.900000,0.976744,0.975734,0.881356,0.842105,0.913793,0.891892
87,87,4,CHEMBL35859,[O][C][=C][C][=C][Branch2][Ring1][Ring2][C][C]...,Oc1ccc(C2CN3CCCC3c3ccccc32)cc1O,1,0,[O][C][=C][C][=C][Branch2][Ring1][Ring2][C][C]...,CN=[S+]=C=CC=CC=CC1CCCN1CCC1=C(C=CO)C1O,CHEMBL35859-1-87-4,-67.418988,0.329154,0.055202,0.442368,0.526316,0.184430,0.123894,0.093960,0.144144,0.108844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388517,388517,48549,CHEMBL3039045,[C][O][C][=C][C][Branch1][C][O][=C][C][=Branch...,COc1cc(O)c2c(=O)c(-c3ccccc3)coc2c1,4,7,[C][O][C][=C][C][Branch1][C][O][=C][C][=Branch...,C=C(C=CC=CC1OC1C)C(=O)C=C(O)C=COC,CHEMBL3039045-4-388517-48549,-70.999891,0.341085,0.056818,0.510460,0.283019,0.104009,0.058824,0.047244,0.048544,0.039062
388518,388518,48550,CHEMBL1365118,[C][O][C][=C][Branch1][=N][N][C][=C][C][=C][Br...,COC1=C(Nc2ccc(O)cc2)C(=O)c2ccccc2C1=O,4,7,[C][O][C][=C][Branch1][P+1][S-1][=N][N][C][=C]...,C#CC=CC=CC(=O)c1pc(O)ccc1NNC([S-])=COC,CHEMBL1365118-4-388518-48550,-43.816394,0.293948,0.142857,0.476038,0.164179,0.311600,0.128440,0.100719,0.294737,0.224000
388520,388520,48552,CHEMBL3604191,[O][=C][C][=C][Branch1][#Branch2][O][C][=C][C]...,O=C1C=C(Oc2ccccc2)C(=O)c2c(O)cccc21,4,7,[O][=C][C][=C][=N][Branch1][#Branch2][C][C][=C...,O=CC=C=NCC=CC=C1CC(O)=CC1=O,CHEMBL3604191-4-388520-48552,-69.807969,0.191406,0.028846,0.428571,0.369565,0.142127,0.120879,0.094828,0.120879,0.094828
388521,388521,48553,CHEMBL4289859,[C][O][C][=C][C][=C][C][=Branch1][=Branch1][=C...,COc1cc2c3c(c1OC)-c1ccccc1CC3N(C)CC2,4,7,[C][O][C][=C][C][=C][C][=P-1][=Branch1][=Branc...,COC=C1C=C2CCN(C)C3C#CC=CC=CC[P-](=C23)C1=[B-]O,CHEMBL4289859-4-388521-48553,-99.640762,0.348189,0.027650,0.602649,0.350649,0.403188,0.216981,0.163121,0.112069,0.086093


In [18]:
mutated_mol_df_deduplicated_with_syba_fp_sim.to_csv(
    Path("data") / "mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024.csv"
)

---

In [74]:
import numpy as np
import e3fp.fingerprint
import e3fp.fingerprint.fprint
import e3fp.pipeline
from typing import List
from functools import partial


def get_e3fp_from_smiles(
    smiles: str,
) -> List[e3fp.fingerprint.fprint.Fingerprint]:
    fprints = e3fp.pipeline.fprints_from_smiles(
        smiles=smiles,
        name="test",
        save=False,
        confgen_params={
            "num_conf": 20,
            "max_energy_diff": 2.0,
            "rmsd_cutoff": 1e-100,
            "seed": 1,
            "first": 5,
        },
        fprint_params={"first": 5},
    )
    return fprints


def compute_tanimoto_similarity(
    fp1: e3fp.fingerprint.fprint.Fingerprint, fp2: e3fp.fingerprint.fprint.Fingerprint
) -> float:
    intersection_card: int = len(set(fp1.indices) & set(fp2.indices))
    union_card: int = fp1.bit_count + fp2.bit_count - intersection_card

    return intersection_card / union_card


def compute_pairwise_tanimoto_similarity(
    fingerprints_mol1: List[e3fp.fingerprint.fprint.Fingerprint],
    fingerprints_mol2: List[e3fp.fingerprint.fprint.Fingerprint],
) -> np.ndarray[np.float32]:

    return np.array(
        [
            [compute_tanimoto_similarity(fp1, fp2) for fp2 in fingerprints_mol2]
            for fp1 in fingerprints_mol1
        ]
    )


def calculate_e3fp_similarities(mol_df: pd.DataFrame) -> pd.DataFrame:
    mol_df = mol_df.copy()
    mol_df["e3fp"] = np.nan

    unique_base_canonical_smiles = mol_df["base_canonical_smiles"].unique()

    for base_canonical_smiles in tqdm(
        unique_base_canonical_smiles, desc="Calculating E3FP"
    ):
        base_canonical_smiles_e3fp = get_e3fp_from_smiles(smiles=base_canonical_smiles)

        mol_df.loc[
            (mol_df["base_canonical_smiles"] == base_canonical_smiles).values, "e3fp"
        ] = (
            mol_df.loc[
                (mol_df["base_canonical_smiles"] == base_canonical_smiles).values,
                "canonical_smiles",
            ]
            .apply(
                lambda x: compute_pairwise_tanimoto_similarity(
                    fingerprints_mol1=base_canonical_smiles_e3fp,
                    fingerprints_mol2=get_e3fp_from_smiles(smiles=x),
                )
            )
            .values
        )

    return mol_df

In [ ]:
# fp1 = get_e3fp_from_smiles(smiles=mutated_mol_df["base_canonical_smiles"].values[0])
# fp2 = get_e3fp_from_smiles(smiles=mutated_mol_df["canonical_smiles"].values[0])
# compute_pairwise_tanimoto_similarity(fp1, fp2)

2024-12-21 16:53:56,289|INFO|Generating conformers for molecule.
2024-12-21 16:53:57,408|INFO|Generated 5 conformers for molecule.
2024-12-21 16:53:57,409|INFO|Generating fingerprints for test.
2024-12-21 16:53:57,719|INFO|Generated 5 fingerprints for test.
2024-12-21 16:53:57,720|INFO|Generating conformers for molecule.
2024-12-21 16:53:58,757|INFO|Generated 5 conformers for molecule.
2024-12-21 16:53:58,758|INFO|Generating fingerprints for test.
2024-12-21 16:53:59,064|INFO|Generated 5 fingerprints for test.


array([[1.        , 0.43859649, 0.43859649, 0.34375   , 0.30769231],
       [0.43859649, 1.        , 0.51851852, 0.45762712, 0.328125  ],
       [0.43859649, 0.51851852, 1.        , 0.36507937, 0.328125  ],
       [0.34375   , 0.45762712, 0.36507937, 1.        , 0.43548387],
       [0.30769231, 0.328125  , 0.328125  , 0.43548387, 1.        ]])

In [76]:
# mutated_mol_df_ = calculate_e3fp_similarities(mol_df=mutated_mol_df)

---

In [ ]:
import numpy as np
import e3fp.fingerprint
import e3fp.fingerprint.fprint
from typing import List


def compute_tanimoto_similarity(
    fp1: e3fp.fingerprint.fprint.Fingerprint, fp2: e3fp.fingerprint.fprint.Fingerprint
) -> float:
    intersection_card: int = len(set(fp1.indices) & set(fp2.indices))
    union_card: int = fp1.bit_count + fp2.bit_count - intersection_card

    return intersection_card / union_card


def compute_pairwise_tanimoto_similarity(
    fingerprints_mol1: List[e3fp.fingerprint.fprint.Fingerprint],
    fingerprints_mol2: List[e3fp.fingerprint.fprint.Fingerprint],
) -> np.ndarray[np.float32]:

    return np.array(
        [
            [compute_tanimoto_similarity(fp1, fp2) for fp2 in fingerprints_mol2]
            for fp1 in fingerprints_mol1
        ]
    ).max()

In [ ]:
# import e3fp.pipeline


# def get_3dfingerprints_from_selfie(selfie: str):
#     smiles = decoder(selfie)
#     fprints = e3fp.pipeline.fprints_from_smiles(
#         smiles=smiles,
#         name="test",
#         save=False,
#         confgen_params={
#             "num_conf": 20,
#             "max_energy_diff": 2.0,
#             "rmsd_cutoff": 1e-100,
#             "seed": 1,
#             "first": 5,
#         },
#         fprint_params={"first": 5},
#     )
#     return fprints

In [51]:
for idx, selfie in zip(known_inh_df.iloc[i, 3:].index, known_inh_df.iloc[i, 3:]):
    print(idx, selfie)

mutated_selfie_0 [C][C][N][Branch2][Ring2][Ring2][C][C][=C][Branch2][=C][Branch2][Ring1][#Branch1][C][N][C][=Branch1][C][=O][C][=C][S][C][N][=C][NH1][C][=Branch1][C][=O][C][Ring1][#Branch2][=Ring1][#Branch1][C][=C][Ring2][Ring1][Ring2][C][C][C][C][C][C][Ring1][=Branch1]
mutated_selfie_1 [C][C][N][Branch2][Ring2][Ring2][C][C][=C][Branch2][=C][Branch2][Ring1][#Branch1][C][N][C][=Branch1][C][=O][C][=C][S][C][N][=C][NH1][C][=Branch1][C][=O][C][#Branch2][=Ring1][#Branch1][C][=C][Ring2][Ring1][Ring2][C][C][C][C][C][C][Ring1][=Branch1]
mutated_selfie_2 [C][C][N][Branch2][Ring2][Ring2][C][C][=C][Branch2][=C][Ring1][#Branch1][C][N][C][=Branch1][C][=O][C][=C][S][C][N][=C][NH1][C][=Branch1][C][=O][C][#Branch2][=Ring1][#Branch1][C][=C][Ring2][Ring1][Ring2][C][C][C][C][C][C][Ring1][=Branch1]
mutated_selfie_3 [C][C][N][Branch2][Ring2][Ring2][C][C][=C][Branch2][Ring1][#Branch1][C][N][C][=Branch1][C][=O][C][=C][S][C][N][=C][NH1][C][=Branch1][C][=O][C][#Branch2][=Ring1][#Branch1][C][=C][Ring2][Ring1][R

In [ ]:
for i in len(known_inh_df):
    for colname, selfie in zip(
        known_inh_df.iloc[i, 3:].index, known_inh_df.iloc[i, 3:]
    ):
        pass
    # get_3dfingerprints_from_selfie(known_inh_df.iloc[i, 2:]["selfie"])

2024-12-17 19:17:29,943|INFO|Generating conformers for molecule.


2024-12-17 19:17:33,181|INFO|Generated 1 conformers for molecule.
2024-12-17 19:17:33,182|INFO|Generating fingerprints for test.
2024-12-17 19:17:33,351|INFO|Generated 1 fingerprints for test.


[Fingerprint(indices=array([18896410, 129199784, 213651285, 303333938, 377032668, 383418862, 461349714, 486993694, 508814906, 526356276, 532215124, 686138476, 812799716, 901725529, 902084654, 934095650, 1075787868, 1198040189, 1201284522, 1383381404, 1578008818, 1640427936, 1754436793, 1832897666, 1880789740, 1909006891, 1960943735, 1981574228, 2029027268, 2030695032, 2159654467, 2163679333, 2173603134, 2181905361, 2191507067, 2237973330, 2241557507, 2262132048, 2371233780, 2414037118, 2453405156, 2479911905, 2553892988, 2559924118, 2589913468, 2598563471, 2738552880, 2836730662, 2946034640, 2946910469, 2971022856, 2993738749, 3007885020, 3009914575, 3015580601, 3037092534, 3039224209, 3144366647, 3259109773, 3288086954, 3425771563, 3464124681, 3489971099, 3507006588, 3560653707, 3654310210, 3663278791, 3708890104, 3729516233, 3829123185, 3889614195, 3947101849, 3957359453, 3977794153, 3982743085, 4048646124, 4093397018, 4136654300, 4139541666, 4147732995, 4156278315, 4156785130, 42324

---